In [6]:
import astropy.units as u
from astroduet.duet_telescope import load_telescope_parameters
from astroduet.duet_sensitivity import src_rate, bgd_sky_qe_rate, bgd_electronics, calc_exposure
from astroduet.bbmag import bb_abmag_fluence
import numpy as np
from matplotlib import pyplot as plt
from astroduet.bbmag import bb_abmag_fluence, bb_abmag

import astroduet.config as config
from astroduet.background import background_pixel_rate


%load_ext autoreload
%autoreload 2

from astropy.visualization import quantity_support
import matplotlib
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 22}

matplotlib.rc('font', **font)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:

# Band1
# 5-sigma limiting magnitude in 1, 2, and 3 stacked frames.

# Account for the fact that you're co-adding the two frames here:
duet = config.Telescope()


#tot_bgd_rate = bgd_band1 + bgd_band2
bandone = duet.bandpass1
bandtwo = duet.bandpass2
exposure = 300*u.s
print()
siglimit=5
dmag = 0.05
print()
for filter_type in ['B']:
    print(filter_type)
    [bgd_band1, bgd_band2] = background_pixel_rate(duet, low_zodi=True, filter_type=filter_type)
    tot_bgd_rate = bgd_band1

    for nframes in [1]:
        snr = 100
        swiftmag = 20 
        while snr > siglimit:
            swiftmag += dmag
            band1_fluence, band2_fluence = bb_abmag_fluence(duet =duet, swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K, filter_type=filter_type)
            band1_rate = duet.fluence_to_rate(band1_fluence)
            band2_rate = duet.fluence_to_rate(band2_fluence)
            src_rate = band1_rate

            snr = duet.calc_snr(exposure, src_rate, tot_bgd_rate, nint=nframes)
        bbmag1, bbmag2 = bb_abmag(swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K, bandone = bandone, bandtwo=bandtwo) 
        
        print('Band1 {} {}-σ magnitude limit: {}'.format(nframes*exposure, siglimit, bbmag1))
        print('Rates: ', src_rate, tot_bgd_rate)
        print(src_rate*exposure, tot_bgd_rate*exposure)
        print()






B
Band1 300.0 s 5-σ magnitude limit: 21.730542480741892 mag(AB)
Rates:  0.26115657255881525 ph / s 0.01753365098332382 ph / s
78.34697176764458 ph 5.260095294997146 ph



In [8]:

# Band2
# 5-sigma limiting magnitude in 1, 2, and 3 stacked frames.

duet = config.Telescope()


#tot_bgd_rate = bgd_band1 + bgd_band2
bandone = duet.bandpass1
bandtwo = duet.bandpass2
exposure = 300*u.s
print()
siglimit=5
dmag = 0.05
print()
for filter_type in ['B']:
    print(filter_type)
    [bgd_band1, bgd_band2] = background_pixel_rate(duet, low_zodi=True, filter_type=filter_type)
    tot_bgd_rate = bgd_band2

    for nframes in [1]:
        snr = 100
        swiftmag = 20 
        while snr > siglimit:
            swiftmag += dmag
            band1_fluence, band2_fluence = bb_abmag_fluence(duet =duet, swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K, filter_type=filter_type)
            band1_rate = duet.fluence_to_rate(band1_fluence)
            band2_rate = duet.fluence_to_rate(band2_fluence)
            src_rate = band2_rate

            snr = duet.calc_snr(exposure, src_rate, tot_bgd_rate, nint=nframes)
        bbmag1, bbmag2 = bb_abmag(swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K, bandone = bandone, bandtwo=bandtwo)        
        print('Band2 {} {}-σ magnitude limit: {}'.format(nframes*exposure, siglimit, bbmag2))







B
Band2 300.0 s 5-σ magnitude limit: 21.141980251219504 mag(AB)


In [9]:

# Stacked channels
# 5-sigma limiting magnitude in 1, 2, 3, and 4 stacked frames.

# Account for the fact that you're co-adding the two frames here:
duet = config.Telescope()
duet.read_noise = (2**0.5) * duet.read_noise


#tot_bgd_rate = bgd_band1 + bgd_band2
bandone = duet.bandpass1
bandtwo = duet.bandpass2
exposure = 300*u.s
print()
siglimit=5
dmag = 0.05
print()
for filter_type in ['B']:
    print(filter_type)
    [bgd_band1, bgd_band2] = background_pixel_rate(duet, low_zodi=True, filter_type=filter_type)
    tot_bgd_rate = bgd_band1 + bgd_band2

    for nframes in [1]:
        snr = 100
        swiftmag = 20 
        while snr > siglimit:
            swiftmag += dmag
            band1_fluence, band2_fluence = bb_abmag_fluence(duet =duet, swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K, filter_type=filter_type)
            band1_rate = duet.fluence_to_rate(band1_fluence)
            band2_rate = duet.fluence_to_rate(band2_fluence)
            src_rate = band1_rate + band2_rate

            snr = duet.calc_snr(exposure, src_rate, tot_bgd_rate, nint=nframes)
        bbmag1, bbmag2 = bb_abmag(swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K, bandone = bandone, bandtwo=bandtwo)        
        print('Stacked Bands {} {}-σ magnitude limit: {}'.format(nframes*exposure, siglimit, bbmag1))







B
Stacked Bands 300.0 s 5-σ magnitude limit: 22.030542480741897 mag(AB)


In [11]:
# FOM: Time to 22 ABmag for this configuration for DUET1
# Old FOM for this 
duet = config.Telescope()


bandone = duet.bandpass1
bandtwo = duet.bandpass2

siglimit=5
swiftmag = 20
dmag = 0.05
bbmag1 = 5*u.ABmag
bbmag_target = 22*u.ABmag
while bbmag1 < bbmag_target:
    swiftmag+=dmag
    bbmag1, bbmag2 = bb_abmag(swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K, bandone = bandone, bandtwo=bandtwo)


# Low Zodi, 10-sigma case
[bgd_band1, bgd_band2] = background_pixel_rate(duet, low_zodi=True)
band1_fluence, foo = bb_abmag_fluence(duet = duet, swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K)
band1_rate = duet.fluence_to_rate(band1_fluence)
req_exp = calc_exposure(siglimit, band1_rate, bgd_band1, duet.read_noise, duet.neff)
print('Low-Zodi {}-sig, {} seconds'.format(siglimit, req_exp.value))


# High Zodi, 10-sigma case.
siglimit=10
[bgd_band1, bgd_band2] = background_pixel_rate(duet, high_zodi=True)
band1_fluence, foo = bb_abmag_fluence(duet = duet, swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K)
band1_rate = duet.fluence_to_rate(band1_fluence)
req_exp = calc_exposure(siglimit, band1_rate, bgd_band1, duet.read_noise, duet.neff)
print('High-Zodi {}-sig, {} seconds'.format(siglimit, req_exp.value))

# High Zodi, 5-sigma case
siglimit=5
[bgd_band1, bgd_band2] = background_pixel_rate(duet, high_zodi=True)
band1_fluence, foo = bb_abmag_fluence(duet = duet, swiftmag=swiftmag*u.ABmag, bbtemp=15e3*u.K)
band1_rate = duet.fluence_to_rate(band1_fluence)

req_exp = calc_exposure(siglimit, band1_rate, bgd_band1, duet.read_noise, duet.neff)
print('High-Zodi {}-sig, {} seconds'.format(siglimit, req_exp.value))




Low-Zodi 5-sig, 415.24932433627276 seconds
High-Zodi 10-sig, 1736.3936879277696 seconds
High-Zodi 5-sig, 539.4962128620656 seconds
